## Model 1.0
Powered by [Eleonora Priori](https://www.est-en.unito.it/do/docenti.pl/Alias?eleonora.priori#tab-profilo) and [Pietro Terna](https://terna.to.it/) 


The trick above, useful to avoid scrolling output windows, does not work with recent jupyter versions; use settings.

====================================================================================================

## 1

import libraries \
MPI init \
context and runner definition \
t(), T(), Tc() function definitions \
random number generator rng creation \
initialization of the parameters from yaml file \
memory allocations to manage ghosts


====================================================================================================

In [1]:
import time
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from math import ceil
from typing import Tuple, List, Dict
import numpy as np
import csv
import os
import sys


comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() 

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initialize the default schedule runner, HERE to create the t() function,
# returning the tick value
runner = schedule.init_schedule_runner(comm)

# tick number
def t():
    return int(runner.schedule.tick)

#Initializes the repast4py.parameters.params dictionary with the model input parameters.
params = parameters.init_params("model1.yaml", "")


if os.path.isdir(params["log_file_root"]+"."+str(rank)):
    os.system("rm -R "+params["log_file_root"]+"."+str(rank))  
os.makedirs(params["log_file_root"]+"."+str(rank)) 

#copy in the output folder the starting set of parameters
os.system("cp model1.yaml "+params["log_file_root"]+"."+str(rank)+"/")
os.system("cp firm-features.csv "+params["log_file_root"]+"."+str(rank)+"/")

if rank==0:
    i=0
    while os.path.isdir(params["log_file_root"]+"."+str(rankNum+i)):
        os.system("rm -R "+params["log_file_root"]+"."+str(rankNum+i))
        i+=1
    

    
#moves to the right folder (that you must create and initialize with a firm-features.csv file)
if not os.path.isdir(params["log_file_root"]+"."+str(rank)):
    print("There is no "+params["log_file_root"]+"."+str(rank) + " starting folder!")  
    sys.exit(0)
else: os.chdir(params["log_file_root"]+"."+str(rank))


        
#dentro a home/model1: "ls "+"../"+params["log_file_root"]+"."+str(rankNum+i))

#generate random seed
repast4py.random.init(rng_seed=params['myRandom.seed'][rank]) #each rank has a seed
rng = repast4py.random.default_rng 


#timer T()
startTime=-1
def T():
    global startTime
    if startTime < 0:
        startTime=time.time()
    return time.time() - startTime
T() #launches the timer

#cpuTimer Tc()
startCpuTime=-1
def Tc():
    global startCpuTime
    if startCpuTime < 0:
        startCpuTime=time.process_time()
    return time.process_time() - startCpuTime
Tc() #launches the cpu timer

agent_cache={} # dict with uid as keys and agents' tuples as values


===================================================================================================

## 2

create agents' classes and restore_agent function 



===================================================================================================

In [2]:
# built-here function to check whether at least one item in a list is != 0
def any(iterable):
    for element in iterable:
        if element != 0:
            return True
    return False
    
class Firm(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, labor:int, capital:float, minOrderDuration:int,\
                 maxOrderDuration:int, recipe: float, laborProductivity: float, maxOrderProduction: float,\
                 assetsUsefulLife: float, plannedMarkup: float, orderObservationFrequency: int, productionType: int,\
                 dimensionalClass: int):
        super().__init__(id=local_id, type=Firm.TYPE, rank=rank) #uid
        self.labor=labor
        self.capital=capital
        self.capitalQ= 0
        self.unavailableLabor=0
        self.unavailableCapitalQ=0
        self.minOrderDuration=minOrderDuration
        self.maxOrderDuration=maxOrderDuration
        self.recipe = recipe
        self.laborProductivity=laborProductivity
        self.maxOrderProduction=maxOrderProduction
        self.assetsUsefulLife=assetsUsefulLife
        self.plannedMarkup=plannedMarkup
        self.orderObservationFrequency=orderObservationFrequency
        self.productionType=productionType
        self.dimensionalClass=dimensionalClass
        
        self.lostProduction=0
        self.inventories=0
        self.inProgressInventories=0
        self.appRepository=[] #aPP is aProductiveProcess
        
        self.profits=0
        self.revenues=0
        self.totalCosts=0
        self.totalCostOfLabor=0
        self.totalCostOfCapital=0
        self.addedValue=0
        self.initialInventories=0
        self.capitalQDynamic=0
        self.myBalancesheet=np.zeros((params['howManyCycles'], 18))

        self.movAvQuantitiesInEachPeriod=[]
        self.movAvDurations=[]
        
        self.productiveProcessIdGenerator=0
        
        
    def settingCapitalQ(self, investmentGoodPrices):
        #############pt temporary solution
        #we temporary use this vector with a unique position as there is only one investment good at the moment
        self.priceOfDurableProductiveGoodsPerUnit = 1 #investmentGoodPrices[0]
        self.currentPriceOfDurableProductiveGoodsPerUnit = 1 #investmentGoodPrices[0]  # the price to be paid to acquire 
                                                                                    # new capital in term of quantity
            
        #pt TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP
        #############   underlying idea:
        #               the actual initial price of durable productive goods (per unit of quantity) must be
        #               consistent with the initial cost of production of the durable productive goods;
        #
        #               the recipe set the ratio K/L where K is expressed in value;
        #
        #               having a price we implicitly set the "quantity";
        #
        #               substitution costs will consider both the change of the quantity and of the price
        #               at which the firm will pay the new productive goods;
        #
        #               the used v. unused capital measures are calculated as addenda of the capital in quantity
        #
        #               the costOfCapital (ratio of interests or rents) will be applied to the current value
        #               of the capital, after calculating the changes in quantity and then in value (considering 
        #               changes in q. and their value using the price of the new acquisitions)
        #
        #               as it evolves over time, the mean price of durable productive goods is an idiosyncratic
        #               property of the firm
        #
        #               L productivity is expressed in quantity as orders are expressed in quantity 

        self.capitalQ=self.capital/self.priceOfDurableProductiveGoodsPerUnit

    def estimatingInitialPricePerProdUnit(self):

        total =  (1/self.laborProductivity)*params['wage']
        total += (1/self.laborProductivity)*self.recipe*params['costOfCapital']/params['timeFraction']
        total += (1/self.laborProductivity)*self.recipe/(self.assetsUsefulLife * params['timeFraction']) 
        if params['usingMarkup']: total *= (1+self.plannedMarkup)
        total *= ((self.maxOrderDuration+self.minOrderDuration)/2)
        return total
    
        
    def dealingMovAvElements(self, freq, x, y):
        
        self.movAvQuantitiesInEachPeriod.append(x/y)
        if len(self.movAvQuantitiesInEachPeriod) > freq: self.movAvQuantitiesInEachPeriod.pop(0) 
            
        self.movAvDurations.append(y)
        if len(self.movAvDurations) > freq: self.movAvDurations.pop(0)

        
    def receivingNewOrder(self, productionOrder: float, orderDuration):

        #creates a statistics of the values of the received order
        self.dealingMovAvElements(self.orderObservationFrequency, productionOrder, orderDuration)
        
        #decision on accepting or refusing the new order
        productionOrderQuantityByPeriod=productionOrder/orderDuration
        requiredLabor=np.ceil(productionOrderQuantityByPeriod/self.laborProductivity)
        requiredCapitalQ=requiredLabor*self.recipe/self.priceOfDurableProductiveGoodsPerUnit
        #if self.uid[0]==29 and self.uid[2]==2:
        #    print("***1",t(),"new order q. per period", productionOrderQuantityByPeriod,\
        #          "req L", requiredLabor, "L", self.labor)
        
        #create a new aPP or skip the order
        if requiredLabor <= self.labor and requiredCapitalQ <= self.capitalQ: 
            self.productiveProcessIdGenerator += 1
            productiveProcessId=(self.uid[0],self.uid[1],self.uid[2],self.productiveProcessIdGenerator)
            aProductiveProcess = ProductiveProcess(productiveProcessId,productionOrderQuantityByPeriod, \
                                                   requiredLabor, requiredCapitalQ, orderDuration,\
                                                   self.priceOfDurableProductiveGoodsPerUnit,\
                                                   self.assetsUsefulLife)
            self.appRepository.append(aProductiveProcess)


    def produce(self)->tuple: 
        
        #total values of the firm in the current interval unit
        self.currentTotalCostOfProductionOrder=0
        self.currentTotalOutput=0
        self.currentTotalCostOfUnusedFactors=0
        self.currentTotalLostProduction=0
        self.currentTotalCostOfLostProduction=0
        
        avgRequiredLabor=0
        avgRequiredCapitalQ=0
        
        if t()==0: self.initialInventories=0 
        else: self.initialInventories=self.inventories+self.inProgressInventories

        # activity within a time unit
        for aProductiveProcess in self.appRepository:  

            if not aProductiveProcess.hasResources and \
                        (self.labor - self.unavailableLabor >= aProductiveProcess.requiredLabor and\
                         self.capitalQ - self.unavailableCapitalQ >= aProductiveProcess.requiredCapitalQ):
                self.unavailableLabor += aProductiveProcess.requiredLabor
                self.unavailableCapitalQ += aProductiveProcess.requiredCapitalQ
                aProductiveProcess.hasResources = True 
                    
            if aProductiveProcess.hasResources: #resources may be just assigned above
                #production
                (aPPoutputOfThePeriod, aPPrequiredLabor, aPPrequiredCapitalQ, aPPlostProduction,\
                 aPPcostOfLostProduction) = aProductiveProcess.step()
                     
                self.currentTotalOutput += aPPoutputOfThePeriod
                
                cost = aPPrequiredLabor*params['wage'] \
                       + aPPrequiredCapitalQ*self.priceOfDurableProductiveGoodsPerUnit \
                                                         *params['costOfCapital']/params['timeFraction']\
                       + aPPrequiredCapitalQ*self.priceOfDurableProductiveGoodsPerUnit/ \
                         (self.assetsUsefulLife * params['timeFraction'])             
                                                       
                self.currentTotalCostOfProductionOrder += cost
                
                self.currentTotalLostProduction += aPPlostProduction
                self.currentTotalCostOfLostProduction += aPPcostOfLostProduction               
        
                if aProductiveProcess.failure:
                    self.inProgressInventories -= cost*(aProductiveProcess.productionClock-1)
                    #NB this is an approximation because in multiperiodal production processes the
                    #   priceOfDurableProductiveGoodsPerUnit may change, but it is a realistic
                    #   approximation in firm accounting
                
                else:
                    if aProductiveProcess.productionClock < aProductiveProcess.orderDuration:
                        self.inProgressInventories += cost
                    else:
                        self.inventories+=cost*aProductiveProcess.orderDuration
                        self.inProgressInventories -= cost*(aProductiveProcess.orderDuration-1)

        self.currentTotalCostOfUnusedFactors =  (self.labor - self.unavailableLabor)*params['wage'] + \
                                        (self.capitalQ - self.unavailableCapitalQ)*\
                                         self.priceOfDurableProductiveGoodsPerUnit*\
                                         params['costOfCapital']/params['timeFraction'] + \
                                         (self.capitalQ - self.unavailableCapitalQ) *\
                                            self.priceOfDurableProductiveGoodsPerUnit/ \
                                            (self.assetsUsefulLife * params['timeFraction'])
                                         # considering substitutions also for the idle capital
        
        #print("ORDER MOV AV",self.uid, sum(self.movAvQuantitiesInEachPeriod)/ len(self.movAvQuantitiesInEachPeriod), flush=True)
        avgRequiredLabor=np.ceil( ((sum(self.movAvQuantitiesInEachPeriod)/len(self.movAvQuantitiesInEachPeriod)) /self.laborProductivity )\
                *( sum(self.movAvDurations)/ len(self.movAvDurations) ))
        
        #total cost of labor
        self.totalCostOfLabor= self.labor*params['wage']
        
        #labor adjustments (frequency at orderObservationFrequency)
        if t() % self.orderObservationFrequency == 0 and t() > 0:
            if self.labor > (1+params['tollerance']) * avgRequiredLabor:
                self.labor = np.ceil((1+params['tollerance']) * avgRequiredLabor) #max accepted q. of L (firing)
            if self.labor < (1/(1+params['tollerance'])) * avgRequiredLabor:
                self.labor = np.ceil((1/(1+params['tollerance'])) * avgRequiredLabor) #min accepted q. of L (hiring)
            #if self.uid==(32,0,0): print("***",self.uid, "labM",avgRequiredLabor,"L", self.labor, flush=True)
           
        
        #capital adjustments (frequency at each cycle)
        self.capitalBeforeAdjustment=self.capital
        capitalQsubstitutions=0
        capitalSubstitutions=0
        requiredCapitalQincrement=0
        requiredCapitalIncrement=0
        if t() > self.orderObservationFrequency: #no corrections before the end of the first correction interval
                                                 #where orders are under the standard flow of the firm
            capitalQmin= self.capitalQ/(1+params['tollerance'])
            capitalQmax= self.capitalQ*(1+params['tollerance'])
            
            avgRequiredCapital=avgRequiredLabor*self.recipe
            avgRequiredCapitalQ=avgRequiredCapital/self.currentPriceOfDurableProductiveGoodsPerUnit
            
            requiredCapitalSubstitution=self.capital/(self.assetsUsefulLife * params['timeFraction'])
            requiredCapitalSubstitutionQ=self.capitalQ/(self.assetsUsefulLife * params['timeFraction']) 
            
            #obsolescence  and deterioration effect
            self.capitalQ-=requiredCapitalSubstitutionQ
            self.capital-=requiredCapitalSubstitution
            
            a=(-requiredCapitalSubstitutionQ)
            #A=(-requiredCapitalSubstitution)
            
            #case I
            if avgRequiredCapitalQ < capitalQmin:
                b=avgRequiredCapitalQ-capitalQmin #being b<0
                #quantities
                if b<=a: capitalQsubstitutions=0
                if b>a: capitalQsubstitutions=abs(a)-abs(b)
                #values
                capitalSubstitutions=capitalQsubstitutions*self.currentPriceOfDurableProductiveGoodsPerUnit
            
            #case II
            if capitalQmin <= avgRequiredCapitalQ and avgRequiredCapitalQ <= capitalQmax:
                #quantities
                capitalQsubstitutions=abs(a)
                #values
                capitalSubstitutions=capitalQsubstitutions*self.currentPriceOfDurableProductiveGoodsPerUnit
            
            #case III
            if avgRequiredCapitalQ > capitalQmax:
                #quantities
                capitalQsubstitutions=abs(a)
                requiredCapitalQincrement=avgRequiredCapitalQ-capitalQmax
                #values
                capitalSubstitutions=capitalQsubstitutions*self.currentPriceOfDurableProductiveGoodsPerUnit
                requiredCapitalIncrement=requiredCapitalQincrement*self.currentPriceOfDurableProductiveGoodsPerUnit
                
            self.capitalQDynamic=capitalQsubstitutions+requiredCapitalQincrement    
            self.capitalQ+=self.capitalQDynamic
            self.capital+=capitalSubstitutions+requiredCapitalIncrement
        
        
        #total cost of capital
        self.totalCostOfCapital=self.capitalBeforeAdjustment*params['costOfCapital']/params['timeFraction']\
                                +capitalQsubstitutions*self.currentPriceOfDurableProductiveGoodsPerUnit
           

        # remove concluded aPPs from the list (backward to avoid skipping when deleting)
        for i in range(len(self.appRepository)-1,-1,-1):
            if self.appRepository[i].productionClock == self.appRepository[i].orderDuration: 
                self.unavailableLabor-=self.appRepository[i].requiredLabor
                self.unavailableCapitalQ-=self.appRepository[i].requiredCapitalQ
                del self.appRepository[i]
                
        
        #print(self.uid,"\n", productionOrder,  self.labor, self.capital, self.recipe,\
              #self.laborProductivity,self.orderDuration,"\n",\
              #output, self.currentTotalCostOfProductionOrder, self.currentTotalCostOfUnusedFactors, self.inventories,\
              #self.inProgressInventories, self.currentTotalLostProduction,\
              #self.currentTotalCostOfLostProduction, flush=True)
        
        return(self.currentTotalOutput, self.currentTotalCostOfProductionOrder, self.currentTotalCostOfUnusedFactors,self.inventories,\
               self.inProgressInventories, self.currentTotalLostProduction, self.currentTotalCostOfLostProduction, \
               self.labor, self.capital)
               # labor, capital modified just above

    def receiveSellingOrders(self, shareOfInventoriesBeingSold: float, centralPlannerBuyingPriceCoefficient: float):
        nominalQuantitySold=shareOfInventoriesBeingSold*self.inventories
        self.revenues=centralPlannerBuyingPriceCoefficient*nominalQuantitySold
        self.inventories-=nominalQuantitySold    
         
    def makeBalancesheet(self):
        self.totalCosts= self.currentTotalCostOfProductionOrder + self.currentTotalCostOfUnusedFactors
        """
        if params['usingMarkup']:
            self.inventories *= (1+self.plannedMarkup) #planned because != ex post
            self.inProgressInventories *= (1+self.plannedMarkup) 
        """
        
        self.profits= self.revenues+(self.inventories + self.inProgressInventories)\
                    -self.totalCosts-self.initialInventories 
        self.addedValue=self.profits+self.totalCosts
        
        self.myBalancesheet[t(), 0]=self.dimensionalClass #i.e. row number in firms-features
        
        self.myBalancesheet[t(), 1]=self.initialInventories
        self.myBalancesheet[t(), 2]=self.totalCosts
        if not self.productionType in params["investmentGoods"]: self.myBalancesheet[t(), 3]=self.revenues
        else: self.myBalancesheet[t(), 4]=self.revenues
        self.myBalancesheet[t(), 5]=self.inventories
        self.myBalancesheet[t(), 6]=self.inProgressInventories
        
        self.myBalancesheet[t(), 7]=self.profits
        self.myBalancesheet[t(), 8]=self.addedValue
        self.myBalancesheet[t(), 9]=self.currentTotalOutput
        self.myBalancesheet[t(), 10]=self.currentTotalCostOfProductionOrder
        self.myBalancesheet[t(), 11]=self.currentTotalCostOfUnusedFactors
        self.myBalancesheet[t(), 12]=self.currentTotalLostProduction
        self.myBalancesheet[t(), 13]=self.currentTotalCostOfLostProduction
        self.myBalancesheet[t(), 14]=self.totalCostOfLabor
        self.myBalancesheet[t(), 15]=self.totalCostOfCapital
        self.myBalancesheet[t(), 16]=self.capitalQDynamic
        self.myBalancesheet[t(), 17]=self.productionType
        
        
    
###########################################################################################################################


class ProductiveProcess(): # no core.Agent???
    def __init__(self, productiveProcessId: tuple, targetProductionOfThePeriod:float, requiredLabor:int,\
                 requiredCapitalQ:float, orderDuration:int, priceOfDurableProductiveGoodsPerUnit:float,\
                 assetsUsefulLife:float):
        
        self.targetProductionOfThePeriod=targetProductionOfThePeriod
        self.requiredLabor = requiredLabor
        self.requiredCapitalQ = requiredCapitalQ
        self.orderDuration = orderDuration
        self.productionClock=0
        self.hasResources= False
        self.productiveProcessId=productiveProcessId
        self.priceOfDurableProductiveGoodsPerUnit=priceOfDurableProductiveGoodsPerUnit
        self.assetsUsefulLife=assetsUsefulLife
        
    #def step(self, productionOrder)->tuple:
    def step(self)->tuple:
        
        lostProduction=0
        costOfLostProduction=0
        self.productionClock += 1
        self.failure=False
        
        # production failure
        if params['probabilityToFailProductionChoices'] >= rng.random():
            self.failure=True
            #if self.productiveProcessId[0]==29 and self.productiveProcessId[2]==2:
            #    print("***2",t(),"failure")
            #print("failure",flush=True)
            lostProduction=self.targetProductionOfThePeriod*self.productionClock
            self.targetProductionOfThePeriod=0
            costOfLostProduction=(params['wage']* self.requiredLabor+\
                                       (params['costOfCapital']/params['timeFraction'])* self.requiredCapitalQ*\
                                        self.priceOfDurableProductiveGoodsPerUnit)*self.productionClock+\
                                        (self.requiredCapitalQ*self.priceOfDurableProductiveGoodsPerUnit)/ \
                                        (self.assetsUsefulLife * params['timeFraction']) 
            self.orderDuration = self.productionClock   

        return(self.targetProductionOfThePeriod, self.requiredLabor, self.requiredCapitalQ, \
               lostProduction, costOfLostProduction)

===================================================================================================

## 3

the model

===================================================================================================

In [3]:
class Model:
    
    global params
    PARAMS = params
    
    def __init__(self, params: Dict):
        
        self.totalProduction=[]
        self.totalCostOfProduction=[]
        self.totalCostOfUnusedFactors=[]
        self.totalInventories=[]
        self.totalInProgressInventories=[]
        self.totalLostProduction=[]
        self.totalCostOfLostProduction=[]
        self.updatedLabor=[]
        self.updatedCapital=[]
        
        #the context and the runner are created in step 1 
      
        runner.schedule_event(          0.0,     self.initInvestmentGoodPrices) 
        runner.schedule_event(          0.0,     self.initGhosts) 
        
        runner.schedule_repeating_event(0.0,  1, self.counter)
        runner.schedule_repeating_event(0.1,  1, self.firmsProducing)
        runner.schedule_repeating_event(0.11, 1, self.firmsMakingFinancialTransactionsRelatedToCosts)
        runner.schedule_repeating_event(0.2,  1, self.firmsReceivingSellingOrders)
        runner.schedule_repeating_event(0.21, 1, self.firmsMakingFinancialTransactionsRelatedToRevenues)
        runner.schedule_repeating_event(0.3,  1, self.enterprisesMakingBalancesheet) #enterprises=firms+banks
        
        runner.schedule_stop(params['howManyCycles'])
        runner.schedule_end_event(self.finish)
        
        ####################################################################################################
        ###################################### CREATE FIRM AGENTS ##########################################
        ####################################################################################################
        
        #importing csv file containing info about firms 
        #share of firms of that class, L min, L max, K min, K max, order duration min, order duration max, 
        #recipe, L prod, max order production, assets' useful life, planned markup, 
        #order observation frequency min, order observation frequency max, production type
        with open('firm-features.csv', newline='') as csvfile:
            firmReader= csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
            
            self.rowNumber=-1
            k=0
            #for each record in firm-features.csv
            #share of firms of that class, L min, L max, K min, K max, order duration min, order duration max, recipe, 
            #L prod, max order production, assets' useful life, planned markup, order observation frequency min, 
            #order observation frequency max, production type
            for row in firmReader:
                print(row)
                if self.rowNumber>=0:
                  for i in range(int(row[0] * params['Firm.count'])// rankNum):
                    labor= rng.integers(row[1], row[2]+1) #+1 because integers exclude extremes
                    capital= row[3] + rng.random()*(row[4] -row[3])
                    minOrderDuration= row[5]
                    maxOrderDuration= row[6]
                    recipe= row[7] #K/L 
                    laborProductivity= row[8]
                    maxOrderProduction= row[9]
                    avgAssetsUsefulLife=row[10]  #https://www.oecd.org/sdd/productivity-stats/43734711.pdf
                    plannedMarkup=row[11]
                    orderObservationFrequency=rng.integers(row[12], row[13]+1)
                    productionType=int(row[14]) #productionType in firm-features.csv indicates the production of
                                                #investment goods if it is into the investmentGoods list in yaml
                    dimensionalClass=int(self.rowNumber)
                    aFirm =Firm(k, rank, labor, capital, minOrderDuration, maxOrderDuration, recipe, laborProductivity,\
                                maxOrderProduction, avgAssetsUsefulLife, plannedMarkup, orderObservationFrequency, productionType,\
                                dimensionalClass)
                    context.add(aFirm)
                    k += 1 # first element of the UID of the agents
                self.rowNumber += 1
  
        
    #initialize investment good prices
    def initInvestmentGoodPrices(self):
        self.investmentGoodPrices=[0]*len(params['investmentGoods'])
        
        for anInvGoodType in range(len(params['investmentGoods'])):
            count=0
            for aFirm in context.agents(agent_type=0):
                if aFirm.productionType == params['investmentGoods'][anInvGoodType]:
                    self.investmentGoodPrices[anInvGoodType]+=aFirm.estimatingInitialPricePerProdUnit()
                    count+=1
            if count != 0: self.investmentGoodPrices[anInvGoodType]/=count
        
        if not any(self.investmentGoodPrices): 
            print("\nThere are no investment goods!")
            sys.exit(0)
        
        for aFirm in context.agents(agent_type=0):
            aFirm.settingCapitalQ(self.investmentGoodPrices)
            if aFirm.uid[0]==0: print("rank",rank,"Initial price of durable productive goods per unit",\
                                      aFirm.priceOfDurableProductiveGoodsPerUnit, flush=True) #as an info to the user
                
                
    #initialize ghosts by sending them in the ranks before starting the simulation 
    def initGhosts(self):
        pass
    
    #count the cycles number
    def counter(self):
        if int(t()) % params["tickNumber.betweenChecks"] == 0: 
            print("rank", rank, "tick", t(), flush=True)
            
  
    
    def firmsProducing(self):
        self.totalProduction.append([0]*(self.rowNumber)) #for each cycle adds a sub-list of lenght number of dimClass
        self.totalCostOfProduction.append([0]*(self.rowNumber))
        self.totalCostOfUnusedFactors.append([0]*(self.rowNumber))
        self.totalInventories.append([0]*(self.rowNumber))
        self.totalInProgressInventories.append([0]*(self.rowNumber))
        self.totalLostProduction.append([0]*(self.rowNumber))
        self.totalCostOfLostProduction.append([0]*(self.rowNumber))
        self.updatedLabor.append([0]*(self.rowNumber))
        self.updatedCapital.append([0]*(self.rowNumber))
        
        for aFirm in context.agents(agent_type=0):
            #print(aFirm.dimensionalClass)
            aFirm.receivingNewOrder(rng.random()*aFirm.maxOrderProduction,\
                    rng.integers(aFirm.minOrderDuration, aFirm.maxOrderDuration+1))
            tupleOfProductionResults = aFirm.produce()

            self.totalProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[0]
            self.totalCostOfProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[1]
            self.totalCostOfUnusedFactors[t()][aFirm.dimensionalClass] += tupleOfProductionResults[2]
            self.totalInventories[t()][aFirm.dimensionalClass] += tupleOfProductionResults[3]
            self.totalInProgressInventories[t()][aFirm.dimensionalClass] += tupleOfProductionResults[4] 
            self.totalLostProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[5]
            self.totalCostOfLostProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[6]
            self.updatedLabor[t()][aFirm.dimensionalClass] += tupleOfProductionResults[7]
            self.updatedCapital[t()][aFirm.dimensionalClass] += tupleOfProductionResults[8]

    def firmsMakingFinancialTransactionsRelatedToCosts(self):
        pass
     
    def firmsReceivingSellingOrders(self):
        for aFirm in context.agents(agent_type=0):
            shareOfInventoriesBeingSold=0.5 +rng.random()*0.5
            centralPlannerBuyingPriceCoefficient = 1.5 #0.8 + rng.random()*0.4
            aFirm.receiveSellingOrders(shareOfInventoriesBeingSold, centralPlannerBuyingPriceCoefficient)

    def firmsMakingFinancialTransactionsRelatedToRevenues(self):
        pass
    
    def enterprisesMakingBalancesheet(self):
        for aFirm in context.agents(agent_type=0):
            aFirm.makeBalancesheet()
      
    
    #finish
    def finish(self):
        
        print("cpu time - calculating phase", Tc(), "rank", rank, flush=True)
        
        # infos for data_analysis.ipynb
        with open('plotInfo.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow((params["log_file_root"],rankNum,\
                             context.size(agent_type_ids=[0])[0]))
        
        """
        print("\n total production", self.totalProduction, flush=True)
        print("\n total cost of production", self.totalCostOfProduction, flush=True)
        print("\n total cost of unused factors", self.totalCostOfUnusedFactors, flush=True)
        print("\n total inventories", self.totalInventories, flush=True)
        print("\n total in progress inventories", self.totalInProgressInventories, flush=True)
        print("\n total lost production", self.totalLostProduction, flush=True)
        print("\n total cost of lost production", self.totalCostOfLostProduction, flush=True)     
        """
        
        names=["_total_production_","_total_cost_of_production_","_total_cost_of_unused_factors_",
               "_total_inventories_","_total_in_progress_inventories_",
               "_total_lost_production_","_total_cost_of_lost_production_","_updatedLabor_","_updatedCapital_"]
        contents=[self.totalProduction,self.totalCostOfProduction,
                  self.totalCostOfUnusedFactors,
                  self.totalInventories,self.totalInProgressInventories,
                  self.totalLostProduction,self.totalCostOfLostProduction,
                  self.updatedLabor,self.updatedCapital]
        
        for s in range(len(names)):
            with open(params["log_file_root"]+names[s]+str(rank)+'.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                for k in range(params["howManyCycles"]):
                    writer.writerow(contents[s][k])
                    
        fileList=[] 
        
        for aFirm in context.agents(agent_type=0):        
            #save balancesheets
            with open(params["log_file_root"]+'_'+str(aFirm.uid)+'.csv', 'w', newline='') as file:
                np.savetxt(file, aFirm.myBalancesheet, delimiter=",")
                #create the list of the files containing the balancesheets
                fileList.append(params["log_file_root"]+'_'+str(aFirm.uid)+'.csv')
        
        #save list of balancesheets file names
        with open(params["log_file_root"]+'_balancesheetList_'+str(rank)+'.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(fileList)
        
        print("cpu time - finishing phase", Tc(), "rank", rank, flush=True)
        print("THE END!", flush=True)
    
    def start(self):
        runner.execute()

=========================================================================================================

## 4

run the model

### if multi-rank, from the terminal launch: mpirun -n x ipython model1.0.ipynb  

where x is the number fo ranks

==========================================================================================================

In [4]:
def run(params: Dict):
    
    model = Model(params) 
    model.start()
    
run(params)

["share of firms of that class, L min, L max, K min, K max, order duration min, order duration max, recipe, L prod, max order production, assets' useful life, planned markup, order observation frequency min, order observation frequency max, production type "]
[0.1, 50.0, 80.0, 6000.0, 9000.0, 5.0, 10.0, 120.0, 0.8, 100.0, 12.0, 0.3, 15.0, 20.0, 1.0]
[0.15, 30.0, 49.0, 3000.0, 4900.0, 5.0, 10.0, 100.0, 0.8, 100.0, 12.0, 0.3, 15.0, 20.0, 0.0]
[0.25, 10.0, 29.0, 800.0, 2320.0, 2.0, 4.0, 80.0, 0.7, 50.0, 12.0, 0.2, 10.0, 15.0, 0.0]
[0.5, 2.0, 9.0, 100.0, 450.0, 1.0, 1.0, 50.0, 0.6, 10.0, 12.0, 0.1, 5.0, 10.0, 0.0]
rank 0 Initial price of durable productive goods per unit 1
rank 0 tick 0
rank 0 tick 10
rank 0 tick 20
rank 0 tick 30
rank 0 tick 40
rank 0 tick 50
rank 0 tick 60
rank 0 tick 70
rank 0 tick 80
rank 0 tick 90
rank 0 tick 100
cpu time - calculating phase 0.6911660000000004 rank 0
cpu time - finishing phase 1.0191209999999997 rank 0
THE END!
